Resources
Style GAN opensource repo
https://github.com/NVlabs/stylegan2-ada-pytorch
How to build a GAN
https://www.youtube.com/watch?v=AALBGpLbj6Q&t=813s&ab_channel=NicholasRenotte
Intro to geemap and Google Earth Engine
https://www.youtube.com/watch?v=wGjpjh9IQ5I&list=PLAxJ4-o7ZoPccOFv1dCwvGI6TYnirRTg3&ab_channel=QiushengWu

Google Earth Engine Python Documentation
https://developers.google.com/earth-engine/guides/python_install
Google Earth Engine sample workflow
https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_AI_Platform.ipynb

# 1. Import Dependencies

We'll be using Matplotlib, Tensorflow, and Numpy to create our models

1. Install these necessary libraries

In [12]:
!pip install earthengine-api --upgrade

  Using cached earthengine_api-0.1.334-py3-none-any.whl
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached google_api_python_client-2.68.0-py2.py3-none-any.whl (10.6 MB)
  Using cached google_cloud_storage-2.6.0-py2.py3-none-any.whl (105 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Using cached google_auth-2.14.1-py2.py3-none-any.whl (175 kB)
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.13.0
    Uninstalling google-auth-2.13.0:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '__init__.py'
Consider using the `--user` option or check the permissions.



In [2]:
!pip install folium

  Using cached folium-0.13.0-py2.py3-none-any.whl (96 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)


In [13]:
!pip install geehydro

  Using cached geehydro-0.2.0-py2.py3-none-any.whl
  Using cached earthengine_api-0.1.334-py3-none-any.whl
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached google_api_python_client-2.68.0-py2.py3-none-any.whl (10.6 MB)
  Using cached google_cloud_storage-2.6.0-py2.py3-none-any.whl (105 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Using cached google_auth-2.14.1-py2.py3-none-any.whl (175 kB)
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.13.0
    Uninstalling google-auth-2.13.0:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '__init__.py'
Consider using the `--user` option or check the permissions.



2. Import these libraries

In [10]:
import tensorflow as tf

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [6]:
import os

In [7]:
import numpy as np

This is the import for Google Earth Engine. It will require a login for access to the data

In [9]:
import ee
ee.Authenticate()
ee.Initialize()
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

ModuleNotFoundError: No module named 'ee'

In [11]:
import folium
from folium import plugins
import geehydro

ModuleNotFoundError: No module named 'geehydro'

# 2. Define Variables

In [ ]:
# Your Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'username'

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'your-bucket'

# Use Landsat 8 surface reflectance data for predictors.
DATASET = ee.ImageCollection('ORTHO/Switzerland/SWISSIMAGE/10cm')
# Use these bands for prediction.
BANDS = ['R', 'G', 'B']

# This is a trianing/testing dataset of points with known land cover labels.
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')
# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landcover'
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 3

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'
# Export imagery in this region.
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])
# The name of the Earth Engine asset to be created by importing
# the classified image from the TFRecord file in Cloud Storage.
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/Classified_pixel_demo'

# 3. Examine Data

This dataset is of high-resolution satellite imagery of the Earth.

https://developers.google.com/earth-engine/datasets/catalog/SKYSAT_GEN-A_PUBLIC_ORTHO_RGB

In [ ]:
Map2 = folium.Map(location=[14.09, -59.9358], zoom_start=3)
Map2.setOptions('HYBRID')

In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(location=[37.5010, -122.1899], zoom_start=10)

# Add the image layer to the map and display it.
map_l8.add_ee_layer(image, image_viz_params, 'false color composite')
display(map_l8)

In [ ]:
dataset = ee.ImageCollection("LANDSAT/LE07/C02/T1")
colorRGB = dataset.select(['B1', 'B2', 'B3', 'B4'])
colorRGBVis = {}
Map2.setCenter(6.746, 46.529, 6)
Map2.addLayer(colorRGB, colorRGBVis, 'SkySat RGB')

In [ ]:
Map2.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map2

1. Create an interactive map

In [ ]:
Boulder = [40.0150, -105.2705]

In [ ]:
Map = folium.Map(location=Boulder, zoom_start=12)
Map.setOptions('HYBRID')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

Map

In [ ]:
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=7uSjLbh1LBeX

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
            print('Rendering Image')
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
image = (ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB')
         .filterBounds(ee.Geometry.Point(Boulder[1], Boulder[0]))
         # .filterDate('2014-01-01', '2016-12-31')
         .first())

Map2 = folium.Map(location=[47.3653, 8.5319], zoom_start=11)

# Add the image layer to the map and display it.
Map2.add_ee_layer(
    image, {'bands': ['R', 'G', 'B'], 'min': 1, 'max': 255}, 'image')
display(Map2)

In [ ]:
# A function provided by Google to import data from a dataset as tensors to use in Tensorflow models
ee.Model.fromAiPlatformPredictor()

In [ ]:
# An image collection dataset of satellite images of Switzerland
imageCollection = ee.ImageCollection("ORTHO/Switzerland/SWISSIMAGE/10cm")
Bern = [46.9480, 7.4474]

image = (imageCollection
         .filterBounds(ee.Geometry.Point(Bern[1], Bern[0]))
         .filterDate('2017-01-01', '2021-12-31')
         # .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())

Map = folium.Map(location=Bern, zoom_start=11)

# Add the image layer to the map and display it.
Map.add_ee_layer(
    image, {'bands': ['R', 'G', 'B'], 'min': 1, 'max': 255}, 'image')
display(Map)

In [ ]:
dataset = ee.ImageCollection("ORTHO/Switzerland/SWISSIMAGE/10cm")
colorRGB = dataset.select(['R', 'G', 'B'])
colorRGBVis = {}
Map.setCenter(Bern[0], Bern[1])
Map.addLayer(colorRGB, colorRGBVis, 'SkySat RGB')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

# 3. Build Neural Network

Should we use a DNN (https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb) or a CNN (https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb)?

In [ ]:
def build_gen_model():
    model = Sequential()
    
    # takes in random values and builds 3D array of 7 x 7 x 128
    model.add(Dense(7*7*128, input_dim=128))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7, 7, 128)))
    
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))
    
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))
    
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))
    
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))
    
    model.add(Conv2D(1, 4, padding='same', activation='sigmoid'))
    
    return model

In [ ]:
gen_model = build_gen_model()

In [ ]:
gen_model.summary()

## 3.1 Build Generator

## 3.2 Build Discriminator

# 4. Train the Network

# 5. Test